In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv('../raw_data/train_ancestors.csv')
df.head(10)

,id,ancestor_id,parent_id
0,00001756c60be8,945aea18,NaN
1,00015c83e2717b,aa2da37e,317b65d12af9df
2,0001bdd4021779,a7711fde,NaN
3,0001daf4c2c76d,090152ca,NaN
4,0002115f48f982,272b483a,NaN
5,00035108e64677,a41da3f9,NaN
6,00038c2941faa0,32614c86,NaN
7,00039fc77a9dd1,dab46c7b,NaN
8,00058dc97eae4b,c54154f7,NaN
9,000597ac4c6700,81ab937c,NaN


In [3]:
df.shape

(139256, 3)

In [4]:
df['id'].nunique(), df['ancestor_id'].nunique(), df['parent_id'].nunique()

(139256, 125372, 13630)

In [5]:
df.describe()

,id,ancestor_id,parent_id
count,139256,139256,19542
unique,139256,125372,13630
top,00001756c60be8,4569bfc1,9d45e544c18f00
freq,1,65,58


In [6]:
sorted_df = df.groupby('ancestor_id')['id'].count().sort_values(ascending=False).reset_index()
sorted_df.columns = ['ancestor_id', 'count']
sorted_df

,ancestor_id,count
0,4569bfc1,65
1,51dd4f97,49
2,021526f8,44
3,09489e94,43
4,612efa48,41
...,...,...
125367,57aaa017,1
125368,57aa437b,1
125369,57aa0621,1
125370,57a9b26d,1


In [7]:
plt.figure(figsize=(10, 6))
sns.histplot(x='ancestor_id', y='count', data=sorted_df)
plt.title('Distribution of Ancestor IDs by Count')
plt.xlabel('Ancestor ID')
plt.ylabel('Count of IDs')
plt.show()

KeyboardInterrupt: 